# Neighborhoods in Dusseldorf, Germany 

## Table of Contents

1. Introduction
2. Describe and Explore Data set
3. Methodology
4. Results
5. Discussion
6. Conclusion


# 1. Introduction

### Definition of the problem
My dream was to open a store or shop one day in the city I'm born in. That city is Ratingen, Germany. 
Currently, I'm working and living in Dusseldorf. As it is now time to open a shop, I can decide to do that in 
Ratingen or in Dusseldorf. Dusseldorf is much bigger than Ratingen. Nevertheless, Ratingen has less shops and 
that's why the chance for success could be bigger. Maybe other cities in the neighborhood of Dusseldorf are better!?

My journey should give insights in environment of these two cities and other cities in the neighborhood. 
The result should be a suggestion where to open a shop. So, the project is interesting for everybody how lives 
near by or wants to open a shop close to Dusseldorf. The project give insights about how many spots of interest 
already exists. E.g. if there are already 10 bakeries in a city, you shouldn't open the 11s bakery if you want to be successful...

Before I start with exploring the data, let's download all the dependencies that we will need.

In [2]:
import numpy as np 

import pandas as pd # library for data analsysis

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

# 2. Description of the data 
I use data from foursquare.com, where I search with the latitude and longitude of both cities as well as the data of their neighborhoods.
I use geopy library to get the latitude and longitude values of Dusseldorf, Ratingen and so on. To get the values for the neighborhoods I set-up a dataframe and use a self-defined function called *getlocation*. The most common venues will show which shops and stores are already very present in each city.

#### Display the geographical data of Dusseldorf and create a *pandas* dataframe with Neighborhoods

#### I use geopy library to get the latitude and longitude values of Dusseldorf and Neighborhood

In [14]:
address = 'Dusseldorf'

geolocator = Nominatim(user_agent="dd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dusseldorf are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dusseldorf are 51.2254018, 6.7763137.


In [15]:
#Set up data frame
Neighborhood=['Ratingen', 'Bochum', 'Cologne', 'Wuppertal', 'Monchengladbach', 'Leverkusen', 
              'Duisburg', 'Neuss', 'Erkrath', 'Krefeld', 'Langenfeld']
df = pd.DataFrame(Neighborhood, columns=['Neighborhood'])

df['Borough']='Dusseldorf'
df = df[['Borough','Neighborhood']]

df['Latitude'] = np.nan
df['Longitude'] = np.nan

In [16]:
#Define function to search for geographical data of location
def getlocation(adress, data):
          
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
        
    df['Latitude'][data]= latitude
    df['Longitude'][data]= longitude

In [17]:
#Put neighborhood data into dateframe
geolocator = Nominatim(user_agent="dd_explorer")

for data in range(0,11):
    address = df['Neighborhood'][data] 
    getlocation(address, data)  

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
df

,Borough,Neighborhood,Latitude,Longitude
0,Dusseldorf,Ratingen,51.297326,6.849350
1,Dusseldorf,Bochum,51.481811,7.219664
2,Dusseldorf,Cologne,50.938361,6.959974
3,Dusseldorf,Wuppertal,51.264018,7.178037
4,Dusseldorf,Monchengladbach,51.194698,6.435364
5,Dusseldorf,Leverkusen,51.032474,6.988119
6,Dusseldorf,Duisburg,51.434999,6.759562
7,Dusseldorf,Neuss,51.198178,6.691648
8,Dusseldorf,Erkrath,51.222206,6.912324
9,Dusseldorf,Krefeld,51.333120,6.562334


#### First result: 
As you can see from the table above, the Latitude and the Longitude of the different cities don't vary much. 
That means, there are lots of cities really close to Dusseldorf. 
So it makes sense to think about where to open a store as you don't have to drive that far.

# 3. Methodology section

As a first step I will visualise the map of Dusseldorf and surroundings to give a better expression about the neighborhood. 
With foursquare.com I will explore the neighborhood of Dusseldorf further. I will search for the main venues in each city. Then I will analyse each neighborhood and cluster the neighborhoods with k-means. Additionally, I will make a test for the best k to be sure to cluster correctly. 

#### Examine the map of Dusseldorf

In [19]:
map_dusseldorf = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dusseldorf)

map_dusseldorf

#### Create a map of Dusseldorf with neighborhoods superimposed on top.

In [20]:
# create map of New York using latitude and longitude values
map_dusseldorf = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dusseldorf)  
    
map_dusseldorf

#### Define Foursquare Credentials and Version (I hid these cells)

In [21]:
# The code was removed by Watson Studio for sharing.

#### Explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [22]:
df.loc[0, 'Neighborhood']

'Ratingen'

Get the neighborhood's latitude and longitude values.

In [23]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ratingen are 51.2973261, 6.8493503.


#### Let's get the top 10 venues that are in Ratingen within a radius of 1 kilometer.

First, let's create the GET request URL.

In [24]:
LIMIT = 10  # limit of number of venues returned by Foursquare API

radius = 1000 # define radius in meters
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# display URL



Send the GET request and examine the results

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0cc9880be7b4001bc55848'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ratingen',
  'headerFullLocation': 'Ratingen',
  'headerLocationGranularity': 'city',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 51.30632610900001,
    'lng': 6.863717012853388},
   'sw': {'lat': 51.28832609099999, 'lng': 6.8349835871466125}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d6e8264134ea1436b8e911a',
       'name': 'Il Gelato',
       'location': {'address': 'Bechemer Straße 23',
        'lat': 51.29630865481922,
        'lng': 6.85085784103195,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.29630865481922,
          '

Let's borrow the **get_category_type** function from the Foursquare lab.

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Il Gelato,Ice Cream Shop,51.296309,6.850858
1,Perfetto am Markt,Coffee Shop,51.297560,6.849489
2,Marktplatz,Plaza,51.296964,6.849832
3,James Joyce Irish Pub,Irish Pub,51.297538,6.848881
4,Rösterei Rosendahl,Café,51.295994,6.849847


<a id='item2'></a>

## Explore Neighborhoods in Dusseldorf

#### Let's create a function to repeat the same process to all the neighborhoods in Dusseldorf

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called *Dusseldorf_venues*.

In [29]:
dusseldorf_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Ratingen
Bochum
Cologne
Wuppertal
Monchengladbach
Leverkusen
Duisburg
Neuss
Erkrath
Krefeld
Langenfeld


#### Let's check the size of the resulting dataframe

In [30]:
print(dusseldorf_venues.shape)
dusseldorf_venues.head()

(110, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ratingen,51.297326,6.84935,Il Gelato,51.296309,6.850858,Ice Cream Shop
1,Ratingen,51.297326,6.84935,Perfetto am Markt,51.297560,6.849489,Coffee Shop
2,Ratingen,51.297326,6.84935,Marktplatz,51.296964,6.849832,Plaza
3,Ratingen,51.297326,6.84935,James Joyce Irish Pub,51.297538,6.848881,Irish Pub
4,Ratingen,51.297326,6.84935,Rösterei Rosendahl,51.295994,6.849847,Café


Let's check how many venues were returned for each neighborhood

In [31]:
dusseldorf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bochum,10,10,10,10,10,10
Cologne,10,10,10,10,10,10
Duisburg,10,10,10,10,10,10
Erkrath,10,10,10,10,10,10
Krefeld,10,10,10,10,10,10
Langenfeld,10,10,10,10,10,10
Leverkusen,10,10,10,10,10,10
Monchengladbach,10,10,10,10,10,10
Neuss,10,10,10,10,10,10


#### Result:
The dataframe has 10 inputs for each city and column. So, lots of data for the neighborhoods exist!

#### Let's find out how many unique categories can be curated from all the returned venues

In [33]:
print('There are {} uniques categories.'.format(len(dusseldorf_venues['Venue Category'].unique())))

There are 49 uniques categories.


# 4. Results section 

<a id='item3'></a>

## Analyze Each Neighborhood

In [34]:
# one hot encoding
dusseldorf_onehot = pd.get_dummies(dusseldorf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dusseldorf_onehot['Neighborhood'] = dusseldorf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dusseldorf_onehot.columns[-1]] + list(dusseldorf_onehot.columns[:-1])
dusseldorf_onehot = dusseldorf_onehot[fixed_columns]

dusseldorf_onehot.head()

,Neighborhood,Argentinian Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Beer Bar,Beer Store,Bookstore,Brewery,Burger Joint,Cable Car,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Drugstore,Fast Food Restaurant,Gas Station,Gastropub,German Restaurant,Greek Restaurant,Grilled Meat Restaurant,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jazz Club,Mongolian Restaurant,Park,Pedestrian Plaza,Pizza Place,Plaza,Pub,Restaurant,Salon / Barbershop,Shopping Mall,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Trattoria/Osteria,Turkish Restaurant
0,Ratingen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ratingen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ratingen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ratingen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ratingen,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [35]:
dusseldorf_onehot.shape

(110, 50)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
dusseldorf_grouped = dusseldorf_onehot.groupby('Neighborhood').mean().reset_index()
dusseldorf_grouped

,Neighborhood,Argentinian Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Beer Bar,Beer Store,Bookstore,Brewery,Burger Joint,Cable Car,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Drugstore,Fast Food Restaurant,Gas Station,Gastropub,German Restaurant,Greek Restaurant,Grilled Meat Restaurant,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jazz Club,Mongolian Restaurant,Park,Pedestrian Plaza,Pizza Place,Plaza,Pub,Restaurant,Salon / Barbershop,Shopping Mall,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Trattoria/Osteria,Turkish Restaurant
0,Bochum,0.0,0.0,0.0,0.2,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1,Cologne,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.2,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Duisburg,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.1,0.0
3,Erkrath,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.1,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Krefeld,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Langenfeld,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.1,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Leverkusen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Monchengladbach,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
8,Neuss,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
9,Ratingen,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Result:
As you can see from the table above, eg. Krefeld has a mean of 0.5 for Cafes, which means that Krefeld already has lots of Cafes. So I would nat recommend to open a new Cafe in Krefeld. 
    

## Count the number of hot spots in all cities

In [37]:
dusseldorf_sum = dusseldorf_grouped.sum(axis = 0)
dusseldorf_sum

Neighborhood               BochumCologneDuisburgErkrathKrefeldLangenfeldL...
Argentinian Restaurant                                                   0.1
Art Museum                                                               0.1
Asian Restaurant                                                         0.3
Bakery                                                                   0.4
Bar                                                                      0.2
Beer Bar                                                                 0.1
Beer Store                                                               0.1
Bookstore                                                                0.2
Brewery                                                                  0.4
Burger Joint                                                             0.1
Cable Car                                                                0.2
Café                                                                     1.3

#### Result:
From the count of the hot spots in all cities, you can see, that there are already a lot of Cafes, Italian Restaurants and Drugstores exist.
But there is only one of the following eg. hotel, Irish Pub or Game Store.

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in dusseldorf_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dusseldorf_grouped[dusseldorf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bochum----
                  venue  freq
0                Bakery   0.2
1            Restaurant   0.1
2  Mongolian Restaurant   0.1
3      Toy / Game Store   0.1
4            Beer Store   0.1


----Cologne----
          venue  freq
0         Plaza   0.2
1  Concert Hall   0.1
2    Art Museum   0.1
3     Jazz Club   0.1
4      Beer Bar   0.1


----Duisburg----
               venue  freq
0    Thai Restaurant   0.2
1               Café   0.2
2     Ice Cream Shop   0.1
3   Asian Restaurant   0.1
4  Trattoria/Osteria   0.1


----Erkrath----
               venue  freq
0             Bakery   0.2
1              Hotel   0.1
2     Ice Cream Shop   0.1
3  German Restaurant   0.1
4        Supermarket   0.1


----Krefeld----
                venue  freq
0                Café   0.5
1  Italian Restaurant   0.2
2           Bookstore   0.1
3           Drugstore   0.1
4   Indian Restaurant   0.1


----Langenfeld----
           venue  freq
0    Pizza Place   0.2
1      Drugstore   0.2
2  Shopping Mall  

#### Let's put that into a *pandas* dataframe

Definition of a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dusseldorf_grouped['Neighborhood']

for ind in np.arange(dusseldorf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dusseldorf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bochum,Bakery,Greek Restaurant,German Restaurant,Toy / Game Store,Beer Store,Bookstore,Restaurant,Coffee Shop,Mongolian Restaurant,Clothing Store
1,Cologne,Plaza,Park,Art Museum,Beer Bar,Steakhouse,Concert Hall,Brewery,Jazz Club,Pedestrian Plaza,Gastropub
2,Duisburg,Thai Restaurant,Café,Asian Restaurant,Trattoria/Osteria,Ice Cream Shop,Indie Movie Theater,Salon / Barbershop,Brewery,Turkish Restaurant,Clothing Store
3,Erkrath,Bakery,Hotel,German Restaurant,Ice Cream Shop,Gas Station,Supermarket,Fast Food Restaurant,Shopping Mall,Drugstore,Chinese Restaurant
4,Krefeld,Café,Italian Restaurant,Indian Restaurant,Bookstore,Drugstore,Chinese Restaurant,German Restaurant,Gastropub,Gas Station,Fast Food Restaurant


#### Result:
As you can see from the table above, Erkrath has lots of good restaurants, but no Cafe under the first 10 common venues. Do you want to open a Cafe in Erkrath?

<a id='item4'></a>

## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters.

In [41]:
# set number of clusters
kclusters = 3

dusseldorf_grouped_clustering = dusseldorf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dusseldorf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1], dtype=int32)

In [42]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dusseldorf_merged = df

# merge to add latitude/longitude for each neighborhood
dusseldorf_merged = dusseldorf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dusseldorf_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dusseldorf,Ratingen,51.297326,6.849350,1,Plaza,Brewery,Ice Cream Shop,Irish Pub,Coffee Shop,Café,Greek Restaurant,Restaurant,German Restaurant,Bar
1,Dusseldorf,Bochum,51.481811,7.219664,1,Bakery,Greek Restaurant,German Restaurant,Toy / Game Store,Beer Store,Bookstore,Restaurant,Coffee Shop,Mongolian Restaurant,Clothing Store
2,Dusseldorf,Cologne,50.938361,6.959974,1,Plaza,Park,Art Museum,Beer Bar,Steakhouse,Concert Hall,Brewery,Jazz Club,Pedestrian Plaza,Gastropub
3,Dusseldorf,Wuppertal,51.264018,7.178037,2,Supermarket,Cable Car,Grilled Meat Restaurant,Trail,Theater,Gas Station,Pub,Café,German Restaurant,Gastropub
4,Dusseldorf,Monchengladbach,51.194698,6.435364,0,Argentinian Restaurant,Shopping Mall,German Restaurant,Coffee Shop,Clothing Store,Café,Pizza Place,Gym / Fitness Center,Asian Restaurant,Sushi Restaurant


Finally, let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dusseldorf_merged['Latitude'], dusseldorf_merged['Longitude'], dusseldorf_merged['Neighborhood'], dusseldorf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [45]:
dusseldorf_merged.loc[dusseldorf_merged['Cluster Labels'] == 0, dusseldorf_merged.columns[[1] + list(range(5, dusseldorf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Monchengladbach,Argentinian Restaurant,Shopping Mall,German Restaurant,Coffee Shop,Clothing Store,Café,Pizza Place,Gym / Fitness Center,Asian Restaurant,Sushi Restaurant
5,Leverkusen,Italian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Drugstore,Shopping Mall,Coffee Shop,Clothing Store,German Restaurant,Gastropub
6,Duisburg,Thai Restaurant,Café,Asian Restaurant,Trattoria/Osteria,Ice Cream Shop,Indie Movie Theater,Salon / Barbershop,Brewery,Turkish Restaurant,Clothing Store
7,Neuss,Café,Brewery,Ice Cream Shop,Gastropub,Italian Restaurant,Pub,Turkish Restaurant,Gym / Fitness Center,Asian Restaurant,Concert Hall
9,Krefeld,Café,Italian Restaurant,Indian Restaurant,Bookstore,Drugstore,Chinese Restaurant,German Restaurant,Gastropub,Gas Station,Fast Food Restaurant


#### Cluster 2

In [46]:
dusseldorf_merged.loc[dusseldorf_merged['Cluster Labels'] == 1, dusseldorf_merged.columns[[1] + list(range(5, dusseldorf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ratingen,Plaza,Brewery,Ice Cream Shop,Irish Pub,Coffee Shop,Café,Greek Restaurant,Restaurant,German Restaurant,Bar
1,Bochum,Bakery,Greek Restaurant,German Restaurant,Toy / Game Store,Beer Store,Bookstore,Restaurant,Coffee Shop,Mongolian Restaurant,Clothing Store
2,Cologne,Plaza,Park,Art Museum,Beer Bar,Steakhouse,Concert Hall,Brewery,Jazz Club,Pedestrian Plaza,Gastropub
8,Erkrath,Bakery,Hotel,German Restaurant,Ice Cream Shop,Gas Station,Supermarket,Fast Food Restaurant,Shopping Mall,Drugstore,Chinese Restaurant
10,Langenfeld,Drugstore,Pizza Place,Plaza,Café,Bar,Shopping Mall,Restaurant,Burger Joint,Gastropub,Gas Station


#### Cluster 3

In [47]:
dusseldorf_merged.loc[dusseldorf_merged['Cluster Labels'] == 2, dusseldorf_merged.columns[[1] + list(range(5, dusseldorf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Wuppertal,Supermarket,Cable Car,Grilled Meat Restaurant,Trail,Theater,Gas Station,Pub,Café,German Restaurant,Gastropub


# 5. Discussion section 

After exploring the data of common venues in cities around Dusseldorf we see a lot of interesting results:
1. First of all, the neighborhood of Dusseldorf is very large. You can see that from the latitude and longitude of the different cities around Dusseldorf. The latitude and the longitude don't differ much.
2. Furthermore, you can observe that Erkrath has lots of good restaurants, but no Cafe under the first 10 common venues. Do you want to open a Cafe in Erkrath?
3. In Duisburg, there are already lots of restaurant or other shops related to food and drink. It seems that there are no famous parks, rivers, or plazas. So, if you like the nature, it is maybe not the best place to move to. But if Duisburg is your preferred city, I recommend to open a concert hall or a gym to entertain the population.
4. Wuppertal seems to be a special city, as it doesn't belong to any other clustered cities. Instead, Wuppertal builds its own cluster.
5. You have also seen, that in all cities already a lot of Cafes and Italian Restaurants exist. So it could make sense to open a store which is not part of the list to be specializied.


# 6. Conclusion section

Finally, what I did was to explore the data of common venues for different cities in the neighborhood of Dusseldorf. As Dusseldorf is known as very large city having already lots of stores, it could be a good option to discover the neighborhood of Dusseldorf. The question is in which city it makes sense to open a store of which typ.
From the observation above, I recommend not to open a Cafe or an Italian Restaurant, as most of the cities have their most common venues here. Go for a Game shop or a Flower Shop, something which is very sheldom in the data set. That makes you unique and increases your prospects for success. 

To find out, why Wuppertal is specific, we need more data of that city. That will be the good task for the next journey...

See you!
